<a href="https://colab.research.google.com/github/AdrianDiazGaxiola/Proyecto-Final/blob/main/Ciclones.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install numpy
!pip install pandas

Se importan las librerías a utilizar

In [3]:
import pandas as pd
import numpy as np

In [8]:
url = "https://github.com/AdrianDiazGaxiola/Proyecto-Final/raw/main/Data/Ciclones.csv"

pd.read_csv(url, header=2, encoding='ISO-8859-1')

,\nN°,\nNombre,\nImpacto en tierra,\nFecha de impacto,Categoría\nmáxima,Categoría \nen impacto,Velocidad \nen impacto (km/h),\nCosta
0,1,ADOLPH,"Chamela - Pto Vallarta, Jal.",1983/05/27,TT,TT,65.0,Pacífico
1,NaN,NaN,"Sta. Cruz, Nay.",1983/05/28,TT,TT,65.0,Pacífico
2,2,ADRIAN,"Villa de Tututepec de Melchor Ocampo, Oax.",1981/06/04,DT,DT,30.0,Pacífico
3,3,AGATHA,"Lázaro Cardenas, Mich.",1971/05/24,H1,H1,140.0,Pacífico
4,NaN,NaN,"30 Km Al Este De Tapachula, Chis.",2010/05/29,TT,TT,75.0,Pacífico
...,...,...,...,...,...,...,...,...
314,195,ZETA,"Inmediaciones de Chemuyil, Q. Roo y a 15 km al...",2020/10/26,H2,H1,130.0,Atlántico
315,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
316,FUENTE: CONAGUA. Servicio Meteorológico Nacion...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
317,Sistema Nacional de Información del Agua (SINA).,NaN,NaN,NaN,NaN,NaN,NaN,NaN
